<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## BPE

In [ ]:
# we use transformer tokenezier to validate my code.
from transformers import AutoTokenizer
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [0]:
#| echo: false
#| output: asis
show_doc(ByteLevelDecoder)

---

### ByteLevelDecoder

>      ByteLevelDecoder ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [0]:
#| echo: false
#| output: asis
show_doc(ByteLevelPreTokenizer)

---

### ByteLevelPreTokenizer

>      ByteLevelPreTokenizer (add_prefix_space:bool=False)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
def test_encode_decode():
    txt = "Hello, world! How  are you?good ."
    toks = ByteLevelPreTokenizer(add_prefix_space=False).pre_tokenize_str(txt)
    assert ByteLevelDecoder().decode(toks) == txt


test_encode_decode()

In [ ]:
def test_pre_tokenizer_against_transformers():
    """Test ByteLevelPreTokenizer against transformers' ByteLevel pre-tokenizer"""
    ByteLevelPreTokenizer(add_prefix_space=False).pre_tokenize_str("Hello, world! How  are you?good .")
    sents = [
        "Hello, world! How  are you?good .",
        "Hello world,  How are you ? Go !",
    ]
    add_prefix_space: bool = False
    trf_pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=add_prefix_space)
    my_pre_tokenizer = ByteLevelPreTokenizer(add_prefix_space=add_prefix_space)
    my_decoder = ByteLevelDecoder()

    for s in sents:
        toks = my_pre_tokenizer.pre_tokenize_str(s)
        trf_toks = trf_pre_tokenizer.pre_tokenize_str(s)
        for t, trf_t in zip(toks, trf_toks):
            assert t[0] == trf_t[0], f"{t} != {trf_t}"
            assert t[1][0] == trf_t[1][0], f"{t} != {trf_t}"
            assert t[1][1] == trf_t[1][1], f"{t} != {trf_t}"

        assert my_decoder.decode(my_pre_tokenizer.pre_tokenize_str(s)) == " " + s if add_prefix_space else s


test_pre_tokenizer_against_transformers()

In [0]:
#| echo: false
#| output: asis
show_doc(BPETokenizer)

---

### BPETokenizer

>      BPETokenizer (vocab_size:int=1000)

*Initialize self.  See help(type(self)) for accurate signature.*

In [0]:
#| echo: false
#| output: asis
show_doc(Encoding)

---

### Encoding

>      Encoding (ids, tokens, offsets:Optional[list]=None,
>                attention_mask:Optional[list]=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
tk = BPETokenizer(vocab_size=50)
text = "A Hello world, this is a test. A new world is coming, Hell yes . I love this"
tk.train(text)
txt_to_enc = "Hello world, I love to test this new thing"

encodings = tk.encode(txt_to_enc)

assert "".join([t for t in encodings.tokens]).replace("Ġ", " ") == txt_to_enc
assert "".join([tk.decode([i]) for i in encodings.ids]) == txt_to_enc
assert "".join([txt_to_enc[s:e] for s, e in encodings.offsets]) == txt_to_enc

encodings = tk.encode("Hello TOTO")
assert encodings.tokens[-5:] == ["Ġ", "[UNK]", "[UNK]", "[UNK]", "[UNK]"]

tk.enable_padding(direction="right", length=15)
padded_encodings = tk.encode("Hello world TOTO")
assert padded_encodings.tokens[-5:] == ["[UNK]", "[PAD]", "[PAD]", "[PAD]", "[PAD]"]

In [ ]:
# example using transformer tokenizer on a new training dataset
tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# tokenizer.enable_padding(length=20, pad_token="[PAD]")
trainer = trainers.BpeTrainer(vocab_size=50, special_tokens=["[PAD]", "[UNK]"])  # pad is 0, unk is 1
tokenizer.train_from_iterator([text], trainer=trainer)
tokenizer.get_vocab_size()
tokenizer.post_processor = processors.ByteLevel(trim_offsets=True)
tokenizer.decoder = decoders.ByteLevel()
tokenizer.enable_padding(length=20, pad_token="[PAD]")
sent_to_encode = "Hello World TOTO"
encodings = tokenizer.encode(sent_to_encode)
encodings.offsets, encodings.tokens, encodings.attention_mask, encodings.ids
# tokenizer.id_to_token(0)
print(encodings.ids), print(encodings.tokens)
# tokenizer.get_vocab()

In [ ]:
# tokenizer.enable_padding(pad_id=2, pad_token="<|PAD|>", length=55)
tokenizer.enable_padding(length=30, pad_token="[PAD]")
tokenizer.encode(txt_to_enc[:50]).attention_mask
tokenizer.encode(txt_to_enc[:50]).tokens
# tokenizer.padding
# tokenizer.get_vocab()
tokenizer.encode(txt_to_enc[:50]).tokens
tokenizer.token_to_id("TOTO")

In [ ]:
tk = BPETokenizer(vocab_size=50)

text = "A Hello world, this is a test. A new world is coming, Hell yes . I love this"
tk.train(text)
text_enc = tk.encode("Hello, I love to test this new thing")
print(tk.decode(text_enc.ids))
print(text_enc)